In [ ]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import tqdm
import tqdm.contrib.concurrent
import re
from collections import namedtuple
import ipywidgets
from IPython.display import display
import itertools

from experimental.beacon_sim.rollout_statistics_pb2 import AllStatistics, RolloutStatistics
from experimental.beacon_sim import plot_trials

import importlib
importlib.reload(plot_trials)

In [ ]:
data_dir = os.path.expanduser("/home/erick/scratch/initial_state_fix/")

In [ ]:
Trial = namedtuple('Trial', ['trial_idx', 'cov_size'])
Instance = namedtuple('Instance', ['instance_idx', 'start_position', 'goal_position', 'path_length', 'plan', 'trials'])
instance_finder = re.compile(r'.*?([0-9]+)\.pb')

def unpack_trials(trials: list[RolloutStatistics]):
    return [Trial(trial_idx=i, cov_size=plot_trials.compute_covariance_size(stat)) for i, stat in enumerate(trials)]

def compute_path_length(stats: AllStatistics):
    current_pos = tuple(stats.local_from_start.translation.data)
    dist_m = 0.0
    for path_idx in stats.plan[1:]:
        if path_idx < 0:
            new_pos = (stats.goal.x, stats.goal.y)
        else:
            new_pos = tuple(stats.road_map.points[path_idx].data)
        
        edge_dist_m = ((current_pos[0] - new_pos[0]) ** 2 + (current_pos[1] - new_pos[1]) ** 2) ** 0.5
        dist_m += edge_dist_m
        current_pos = new_pos
    return dist_m

def load_instance(file_name: str) -> Instance:
    with open(file_name, 'rb') as file_in:
        stats = AllStatistics()
        stats.ParseFromString(file_in.read())
    
    m = instance_finder.search(file_name)
    
    instance_idx = int(m.group(1))
    
    
    
    return Instance(instance_idx=instance_idx,
                    start_position=list(stats.local_from_start.translation.data),
                    goal_position=(stats.goal.x, stats.goal.y),
                    path_length = compute_path_length(stats),
                    plan = list(stats.plan),
                    trials = unpack_trials(stats.statistics))


In [ ]:
def load_road_map_and_world_map(file_path):
    with open(file_path, 'rb') as file_in:
        stats = AllStatistics()
        stats.ParseFromString(file_in.read())
        
    return stats.road_map, stats.world_map_config

In [ ]:
file_paths = [os.path.join(data_dir, file_name) for file_name in os.listdir(data_dir)]

In [ ]:
road_map, world_map = load_road_map_and_world_map(file_paths[0])

In [ ]:
instances = tqdm.contrib.concurrent.process_map(load_instance, file_paths, chunksize=1, )

In [ ]:
instances = sorted(instances, key = lambda x: x.instance_idx)

In [ ]:
def missing_from_configuration(configuration, world_map):
    idxs = []
    for i in range(len(world_map.fixed_beacons.beacons)):
        if configuration & (1 << i):
            idxs.append(world_map.fixed_beacons.beacons[i].id)
    return sorted(idxs)

def plot_environment(road_map, world_map, plan, start_pos, goal_pos, configuration=None):
    if configuration is None:
        configuration = 0
    missing_beacons = missing_from_configuration(configuration, world_map)
        
    # plot the road map
    # Plot the edges
    line_segments = []
    for r in range(road_map.adj.num_rows):
        for c in range(r+1, road_map.adj.num_cols):
            idx = r * road_map.adj.num_cols + c
            if (road_map.adj.data[idx] == 1):
                line_segments.append([tuple(road_map.points[r].data), tuple(road_map.points[c].data)])
    edges = mpl.collections.LineCollection(line_segments, colors=(0.8, 0.8, 0.6, 1.0))
    ax = plt.gca()
    ax.add_collection(edges)
    
    # Plot the plan
    plan_points = []
    for node_idx in plan:
        if node_idx == -1:
            plan_points.append(start_pos)
        elif node_idx == -2:
            plan_points.append(goal_pos)
        else:
            plan_points.append(tuple(road_map.points[node_idx].data))
    plan_x = [pt[0] for pt in plan_points]
    plan_y = [pt[1] for pt in plan_points]
    plt.plot(plan_x, plan_y, 'b')
    
    # Plot the start
    plt.plot(*start_pos, 'md', markersize=10)
    
    # Plot the goal
    plt.plot(*goal_pos, 'g*', markersize=15)
    
    # Plot the nodes
    rm_x = [pt.data[0] for pt in road_map.points]
    rm_y = [pt.data[1] for pt in road_map.points]
    plt.plot(rm_x, rm_y, 'rs')
    
    TEXT_X_OFFSET = 0.2
    TEXT_Y_OFFSET = 0.2
#     for i, pt in enumerate(road_map.points):
#         plt.text(pt.data[0] + TEXT_X_OFFSET, pt.data[1] + TEXT_Y_OFFSET, i)
        
    # Plot the beacons
    beacon_xs = [beacon.pos_x_m for beacon in world_map.fixed_beacons.beacons if beacon.id not in missing_beacons]
    beacon_ys = [beacon.pos_y_m for beacon in world_map.fixed_beacons.beacons if beacon.id not in missing_beacons]
    beacon_ids = [beacon.id for beacon in world_map.fixed_beacons.beacons if beacon.id not in missing_beacons]
    
    for x, y, beacon_id in zip(beacon_xs, beacon_ys, beacon_ids):
        plt.text(x + TEXT_X_OFFSET, y + TEXT_Y_OFFSET, beacon_id)
    
    plt.plot(beacon_xs, beacon_ys, 'b^')
    plt.axis('equal')
    
def plot_covariances(trials, current_trial):
    sorted_trials = sorted(trials, key=lambda trial: trial.cov_size)
    plt.plot([t.cov_size for t in sorted_trials], list(range(len(sorted_trials))))
    sorted_idx = sorted_trials.index(current_trial)
    plt.plot(current_trial.cov_size, sorted_idx, 'g*', markersize=15)
    
def make_plots(fig, trial_idx, instance, trials, road_map, world_map):
    trial = trials[trial_idx]
    plt.figure(fig.number)
    plt.gcf().clear()
    plt.subplot(121)
    plot_environment(
        road_map, world_map, instance.plan, instance.start_position,
        instance.goal_position, configuration=trial.trial_idx)
    plt.title(f'Instance: {instance.instance_idx} Trial: {trial.trial_idx}, Path Length: {instance.path_length: 0.2f}')
    plt.xlabel('X (m)')
    plt.ylabel('Y (m)')
    plt.subplot(122)
    plot_covariances(trials, trial)
    plt.xlabel('$|\Sigma|$')
    plt.ylabel('Count')
    plt.title('CDF of $|\Sigma|$')
    plt.tight_layout()
    
def plot_results(road_map, world_map, instances):
    path_length_order_checkbox = ipywidgets.Checkbox(value=True, description='Sorted Instances?')
    instance_slider = ipywidgets.IntSlider(min = 0, max=len(instances)-1, step=1, description='Instance Idx', value=0)
    sorted_trial_checkbox = ipywidgets.Checkbox(value=True, description = "Sorted Trials?")
    style = {"description_width": "initial"}
    trial_slider = ipywidgets.IntSlider(min=0, max=0, description = "Trial Idx", style=style)
    fig = plt.figure(figsize=(10, 6))
    
    hbox = ipywidgets.HBox()
    
    sorted_instances = sorted(instances, key=lambda x: x.path_length)
    
    def on_instance_change(change):
               
        instance_idx = instance_slider.value
        should_use_sorted_path_length = path_length_order_checkbox.value
        instances_to_use = sorted_instances if should_use_sorted_path_length else instances
        instance = instances_to_use[instance_idx]
        
        trials = instance.trials
        sorted_trials = sorted(trials, key=lambda trial: trial.cov_size)
        
        def on_trial_change(change):
            trial_idx = change["owner"].value
            should_use_sorted = sorted_trial_checkbox.value
            trials_to_use = sorted_trials if should_use_sorted else trials
            make_plots(fig, trial_idx, instance, trials_to_use, road_map, world_map)
        
        trial_slider.max = len(trials) - 1
        trial_slider.description = f"Trial for Instance {instance.instance_idx}"
        trial_slider.unobserve_all()
        trial_slider.observe(on_trial_change, 'value')
        on_trial_change({'owner': trial_slider})
    
    def on_sorted_change(change):
        instance_idx = instance_slider.value
        trial_idx = trial_slider.value
        should_use_sorted_trial = sorted_trial_checkbox.value
        should_use_sorted_path_length = path_length_order_checkbox.value
        
        instances_to_use = sorted_instances if should_use_sorted_path_length else instances
        instance = instances_to_use[instance_idx]
        
        trials = instance.trials
        sorted_trials = sorted(trials, key=lambda trial: trial.cov_size)
        trials_to_use = sorted_trials if should_use_sorted_trial else trials
        
        make_plots(fig, trial_idx, instance, trials_to_use, road_map, world_map)
        
    
    instance_slider.observe(on_instance_change, 'value')
    path_length_order_checkbox.observe(on_sorted_change, 'value')
    sorted_trial_checkbox.observe(on_sorted_change, 'value')
    hbox.children = (path_length_order_checkbox, instance_slider, sorted_trial_checkbox, trial_slider)
    display(hbox)
    on_instance_change({'owner': instance_slider})
    
    

In [ ]:
plot_results(road_map, world_map, instances)

In [ ]:
np.random.normal()

In [ ]:
len(world_map.fixed_beacons.beacons)

In [ ]:
def logsumexp(x):
    c = x.max()
    return c + np.log(np.sum(np.exp(x - c)))

In [ ]:
NUM_BEACONS = 12

rng = np.random.default_rng(0)

possibilities = list(itertools.product([0, 1], repeat=NUM_BEACONS))

beta = 1.0

p_00 = 0.8
log_p_00 = np.log(p_00)

correlation = np.eye(NUM_BEACONS) * np.log(1-p_00)
diag_adj = np.eye(*correlation.shape) * log_p_00
correlation = correlation - diag_adj

off_diag_adj = np.zeros_like(diag_adj)
for r in range(NUM_BEACONS):
    for c in range(r+1, NUM_BEACONS):
        value =  - correlation[r, r] - correlation[c,c] - log_p_00 + 

adj_correlation = correlation - diag_adj

pot = np.zeros(len(possibilities))
for i, p in enumerate(possibilities):
    x = np.array(p)
    pot[i] = x @ adj_correlation @ x + log_p_00

log_normalizer = logsumexp(beta * pot)

pot = np.exp(pot - log_normalizer)

In [ ]:
plt.figure()
plt.plot(pot)


In [ ]:
0.8**10

In [ ]:
log_p_00

In [ ]:
np.log((1-p_00) / p_00)

In [ ]:
adj_correlation